# Setup

In [1]:
import dataset
import models.losses as losses
import tensorflow as tf
from models.metrics import *
import models.cnn_autoencoder_model as cnnmodel
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Define mode constants (replacing tf_estimator.ModeKeys)
class ModeKeys:
    TRAIN = 'train'
    EVAL = 'eval'
    PREDICT = 'predict'

2025-11-19 23:25:50.875520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
tf.config.list_physical_devices('GPU')

W0000 00:00:1763594753.051815   28140 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

In [3]:
hparams = {
    # 数据路径
    'train_path': '../dataset/next_day_wildfire_spread_train*',
    'eval_path': '../dataset/next_day_wildfire_spread_eval*',
    'test_path': '../dataset/next_day_wildfire_spread_test*',
    
    # 特征
    'input_features': ['elevation', 'pdsi', 'NDVI', 'pr', 'sph', 'th', 'tmmn',
                  'tmmx', 'vs', 'erc', 'population', 'PrevFireMask'],
    'output_features': ['FireMask'],
    
    # 方位通道
    'azimuth_in_channel': None,
    'azimuth_out_channel': None,
    
    # 数据和模型参数
    'data_sample_size': 64,
    'sample_size': 32,
    'output_sample_size': 32,
    'batch_size': 128,
    'shuffle': False,
    'shuffle_buffer_size': 10000,
    'compression_type': None,
    'input_sequence_length': 1,
    'output_sequence_length': 1,
    'repeat': False,
    'clip_and_normalize': True,
    'clip_and_rescale': False,
    
    # 数据增强
    'random_flip': False,
    'random_rotate': False,
    'random_crop': False,
    'center_crop': True,
    
    # 其他参数
    'downsample_threshold': 0.0,
    'binarize_output': True
}

train_dataset = dataset.make_dataset(
    hparams,
    mode = ModeKeys.TRAIN
)
val_dataset = dataset.make_dataset(
    hparams,
    mode = ModeKeys.EVAL
)
test_dataset = dataset.make_dataset(
    hparams,
    mode = ModeKeys.PREDICT
)

In [4]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True,
)

# Training

## Autoencoder

In [23]:
input_tensor = Input((32, 32, 12))
num_out_channels = 1
encoder_layers = [16,32]
decoder_layers = [32,16]
encoder_pools = [2,2]
decoder_pools = [2,2]
autoencoder_model = cnnmodel.create_model(
    input_tensor,
    num_out_channels,
    encoder_layers,
    decoder_layers,
    encoder_pools,
    decoder_pools,
)
autoencoder_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 32, 32,    │          0 │ -                 │
│ (InputLayer)        │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 32, 32,    │      1,728 │ input_layer_3[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_9       │ (None, 32, 32,    │          0 │ conv2d_14[0][0]   │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 32, 32,    │        192 │ input_layer_3[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 32, 32,    │          0 │ leaky_re_lu_9[0]… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 32, 32,    │          0 │ conv2d_16[0][0]   │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 32, 32,    │      2,304 │ dropout_14[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 32, 32,    │          0 │ dropout_15[0][0], │
│                     │ 16)               │            │ conv2d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_10      │ (None, 32, 32,    │          0 │ add_5[0][0]       │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 32, 32,    │          0 │ leaky_re_lu_10[0… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ dropout_16[0][0]  │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_11      │ (None, 16, 16,    │          0 │ max_pooling2d_2[… │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 16, 16,    │          0 │ leaky_re_lu_11[0… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 16, 16,    │      4,608 │ dropout_17[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 16, 16,    │        512 │ add_5[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 16, 16,    │          0 │ conv2d_17[0][0]   │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 16, 16,    │          0 │ conv2d_18[0][0],

 Total params: 46,480 (181.56 KB)

 Trainable params: 46,480 (181.56 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
autoencoder_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=3),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = autoencoder_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

Epoch 1/1000


ValueError: `logits` and `labels` must have the same shape, received ((None, 1024) vs (None, 32, 32, 1)).

In [ ]:
autoencoder_model.evaluate(test_dataset)

## ResNet

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
import models.model_utils as model_utils
import models.cnn_autoencoder_model as cnn_autoencoder_model
from models.cnn_autoencoder_model import decoder
from tensorflow.compat.v2 import keras


layers_list = (32, 64, 128, 256, 256)
pools_list = (2, 2, 2, 2, 2)
decoder_layers = tuple(reversed(layers_list))
decoder_pools = tuple(reversed(pools_list))
num_out_channels = 1
l1_regularization = model_utils.L1_REGULARIZATION_DEFAULT
l2_regularization = model_utils.L2_REGULARIZATION_DEFAULT

# define input
conv_input = Input(shape=(32,32,12))

# add extra convolutional layer
conv_output = tf.keras.layers.Conv2D(16, (3, 3), padding='same')(conv_input)

# define resnet encoder
keras_resnet_encoder = ResNet50(weights=None,
                 include_top=False,
                input_shape=(32, 32, 16))

encoder_output = keras_resnet_encoder(conv_output)

# define resnet decoder
# decoder_input_img = Input(shape=keras_resnet_encoder.output_shape[1:])

x = decoder(encoder_output, decoder_layers, decoder_pools)
decoder_output = model_utils.conv2d_layer(
      filters=num_out_channels,
      kernel_size=model_utils.RES_SHORTCUT_KERNEL_SIZE,
      l1_regularization=l1_regularization,
      l2_regularization=l2_regularization)(x)

# keras_resnet_decoder = keras.Model(decoder_input_img, resnet_decoder)

# decoder_output = keras_resnet_decoder(encoder_output)

# define connected model
keras_model = keras.Model(inputs = conv_input, outputs = decoder_output)
keras_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
keras_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=3),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = keras_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

In [ ]:
keras_model.evaluate(test_dataset)

## UNet

In [ ]:
def expend_as(tensor, rep):
     return Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)

def double_conv_layer(x, filter_size, size, dropout, batch_norm=False):
    axis = 3
    conv = SeparableConv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = SeparableConv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    if dropout > 0:
        conv = Dropout(dropout)(conv)

    shortcut = Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = BatchNormalization(axis=axis)(shortcut)

    res_path = add([shortcut, conv])
    return res_path

def encoder(inputs):
    num_filters = [16, 32, 64, 128]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        a = double_conv_layer(x, 3, f, 0.1, True)
        skip_connections.append(a)
        x = MaxPooling2D(pool_size=(2, 2))(a)
    
    return x, skip_connections

def bottleneck(inputs):
    x = inputs
    f = 256
    
    x3 = double_conv_layer(x, 3, f, 0.1, True)
    
    return x3

def decoder(inputs, skip_connections):
    num_filters = [128, 64, 32, 16]
    skip_connections.reverse()
    x = inputs
    batch_norm = True
    
    for i, f in enumerate(num_filters):
        
        x_up = UpSampling2D(size=(2, 2), data_format="channels_last")(x)
        x_att = concatenate([x_up, skip_connections[i]], axis=-1)
        
        x = double_conv_layer(x_att, 3, f, 0.1, True)
    return x

def output(inputs):
    x = Conv2D(1, kernel_size=(1,1))(inputs)
    x = BatchNormalization()(x)
    # x = Activation('sigmoid')(x)
    
    return x

inputs = Input((32, 32, 12))
# s = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
s = inputs
x, skip_1 = encoder(s)
x = bottleneck(x)
x = decoder(x, skip_1)
outputs = output(x)
unet_model = Model(inputs, outputs)
unet_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
unet_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=2),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = unet_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

In [ ]:
unet_model.evaluate(test_dataset)

## ViT

In [5]:
import numpy as np
from glob import glob

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, concatenate

import keras_vision_transformer as create_swin_unet
from keras_vision_transformer.create_swin_unet import swin_unet_2d_base

import keras_vision_transformer.create_swin_unet_add_convolution
from keras_vision_transformer.create_swin_unet_add_convolution import swin_unet_2d_base

filter_num_begin = 128     # number of channels in the first downsampling block; it is also the number of embedded dimensions
depth = 4                  # the depth of SwinUNET; depth=4 means three down/upsampling levels and a bottom level
stack_num_down = 2         # number of Swin Transformers per downsampling level
stack_num_up = 2           # number of Swin Transformers per upsampling level
patch_size = (2, 2)        # Extract 4-by-4 patches from the input image. Height and width of the patch must be equal.
num_heads = [4, 8, 8, 8]   # number of attention heads per down/upsampling level
window_size = [4, 2, 2, 2] # the size of attention window per down/upsampling level
num_mlp = 512              # number of MLP nodes within the Transformer
shift_window=True          # Apply window shifting, i.e., Swin-MSA

# define input size
input_size = (32,32,12)
IN = Input(input_size)

# Base architecture
X = swin_unet_2d_base(IN, filter_num_begin, depth, stack_num_down, stack_num_up,
                      patch_size, num_heads, window_size, num_mlp,
                      shift_window=shift_window, name='swin_unet')

# define output: remove activation function
n_labels = 1
OUT = Conv2D(n_labels, kernel_size=1, use_bias=False, padding='same')(X)

# Model Configuration
keras_model = Model(inputs=IN, outputs=OUT)

keras_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'window_attention_1' (of type WindowAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'window_attention_3' (of type WindowAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'window_attention_5' (of type WindowAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warn

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32,    │          0 │ -                 │
│ (InputLayer)        │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      1,744 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_extract       │ (None, 256, 64)   │          0 │ conv2d[0][0]      │
│ (patch_extract)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_embedding     │ (None, 256, 128)  │     41,088 │ patch_extract[0]… │
│ (patch_embedding)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 256, 128)  │    198,468 │ patch_embedding[… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 256, 128)  │    198,468 │ swin_transformer… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_merging       │ (None, 64, 256)   │    131,072 │ swin_transformer… │
│ (patch_merging)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 64, 256)   │    527,176 │ patch_merging[0]… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 64, 256)   │    527,176 │ swin_transformer… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_merging_1     │ (None, 16, 512)   │    524,288 │ swin_transformer… │
│ (patch_merging)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 16, 512)   │  1,578,056 │ patch_merging_1[… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 16, 512)   │  1,578,056 │ swin_transformer… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_merging_2     │ (None, 4, 1024)   │  2,097,152 │ swin_transformer… │
│ (patch_merging)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 4, 1024)   │  5,252,680 │ patch_merging_2[… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_transformer_b… │ (None, 4, 1024)   │  5,252,680 │ swin_transformer… │
│ (SwinTransformerBl… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_expanding     │ (None, 16, 512)   │  2,097,152 │ swin_transformer… │
│ (patch_expanding)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ swin_unet_concat_0  │ (None, 16, 1024)  │          0 │ patch_expanding[

 Total params: 25,988,728 (99.14 MB)

 Trainable params: 25,988,728 (99.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
keras_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=4),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = keras_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

Epoch 1/1000


2025-11-19 23:26:34.933193: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144


      3/Unknown 142s 41s/step - auc_with_masked_class: 0.0272 - loss: 1.2897

In [ ]:
keras_model.evaluate(test_dataset)